

<H1>Методы интеллектуального анализа текстов</H1>
<H2>Тема 5. Семантический анализ текстов</H2>



### Задание 4:
Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.

Импортируем нужные библиотеки.

In [13]:
! pip install transformers
! pip install ufal.udpipe
! pip install git+https://github.com/IINemo/isanlp.git ufal.udpipe
! wget -q https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/IINemo/isanlp.git to /tmp/pip-req-build-lzjh14f_
  Running command git clone -q https://github.com/IINemo/isanlp.git /tmp/pip-req-build-lzjh14f_


In [168]:
import torch
from transformers import BertTokenizer, BertModel
from isanlp.processor_udpipe import ProcessorUDPipe
from zss import simple_distance, Node
from sklearn.metrics.pairwise import cosine_similarity

In [169]:
examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у них в подвале нашли клад!" ]

In [170]:
udpipe = ProcessorUDPipe('russian-syntagrus-ud-2.5-191206.udpipe') # Подключаем морфологичекий анализатор

In [171]:
def pos_dep_tree(postags, syntax_dep_tree):
    """ Конвертируем результат морфосинтаксического анализа в zss-дерево из частей речи и синтаксических связей
    NOUN -> advmod -> VERB """
    
    root = Node('root')
    pos_nodes = {-1: root}
    for i, postag in enumerate(postags):
        pos_nodes[i] = Node(postag)
    
    for i, dependency_edge in enumerate(syntax_dep_tree):
        # Здесь в зависимости от выбранного формата синтаксических аннотаций заполняем граф значениями
        # <синтаксическое отношение, порядковый номер вершины (токена), порядковый номер его родительской вершины>
        relation = dependency_edge.link_name
        index = i
        parent_index = dependency_edge.parent
        
        relation_node = Node(relation)
        pos_nodes[parent_index].addkid(relation_node)
        relation_node.addkid(pos_nodes[index])
    
    return root

def sent_dep_tree(sent):
    """ Получаем список постегов и список синтаксических аннотаций слов предложения, полученные выбранным анализатором (или анализаторами) """
   
    result = udpipe(sent)
    postags = result['postag'][0]
    syntax_dep_tree = result['syntax_dep_tree'][0]
    
    return pos_dep_tree(postags, syntax_dep_tree)

def dep_tree_similarity(dep1, dep2, smoothing=5.0):
    return smoothing / (smoothing + simple_distance(dep1, dep2))

def sentence_similarity(sent1, sent2, smoothing=5.0):
    return dep_tree_similarity(sent_dep_tree(sent1), sent_dep_tree(sent2), smoothing)

In [172]:
for pair in [[0, 1], [1, 2], [0, 2]]:
    print((examples[pair[0]], examples[pair[1]],
           sentence_similarity(examples[pair[0]], examples[pair[1]])))

('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.05263157894736842)
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у них в подвале нашли клад!', 0.05263157894736842)
('Привет, у нас на кухне нашли плесень!', 'Привет, у них в подвале нашли клад!', 1.0)


In [173]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [174]:
def embed_sentence(sentence: str):
    input_ids = tokenizer(sentence)  # Токенизация
    outputs =  model.forward(torch.tensor([input_ids['input_ids']])) # Предсказания модели
    return torch.mean(outputs[0].detach(), axis=0).numpy()

In [175]:
embedded_examples = [embed_sentence(sent) for sent in examples]

In [176]:
embedded_examples 

[array([[-0.7679099 , -0.39528573, -0.32386765, ..., -0.24590759,
         -1.0639223 ,  0.67690706],
        [-0.6216488 , -0.3756859 , -0.09958875, ..., -0.32701218,
         -1.0036204 ,  0.81859404],
        [-0.69259816, -0.21249032, -0.21213473, ..., -0.3714689 ,
         -1.045823  ,  0.8058061 ],
        ...,
        [-0.7504629 , -0.4188759 , -0.09141711, ...,  0.09274261,
         -1.063937  ,  0.70093745],
        [-0.70663464, -0.3052379 , -0.2598623 , ..., -0.26921755,
         -1.0927025 ,  0.79551506],
        [-0.7382749 , -0.39041314, -0.29134142, ..., -0.27106404,
         -1.1052116 ,  0.74953806]], dtype=float32),
 array([[-0.9968567 ,  0.2967537 , -0.10190902, ..., -0.8692864 ,
         -0.24060129,  1.0921069 ],
        [-0.94869274,  0.16572472,  0.43090323, ..., -0.69339085,
         -0.40038162,  1.0761354 ],
        [-1.1345026 ,  0.03838572,  0.4234135 , ..., -0.96489763,
         -0.38352928,  0.7272421 ],
        ...,
        [-1.106046  ,  0.25641626,  0.6

In [177]:
cos1 = cosine_similarity(embedded_examples[0], embedded_examples[1])
cos2 = cosine_similarity(embedded_examples[0], embedded_examples[2])
cos3 = cosine_similarity(embedded_examples[1], embedded_examples[2])

In [178]:
cos1

array([[0.79197985, 0.7812699 , 0.749678  , 0.7661632 , 0.7735775 ,
        0.7325072 , 0.75032043, 0.7582741 , 0.72470653, 0.75049376,
        0.7259561 , 0.74351764, 0.7748369 , 0.7246938 , 0.7311472 ,
        0.723818  , 0.7440981 , 0.75246865, 0.7423417 , 0.7989063 ,
        0.80019486],
       [0.75969076, 0.7574413 , 0.73650986, 0.7498927 , 0.7585549 ,
        0.7216047 , 0.7368551 , 0.7382963 , 0.7131233 , 0.72616136,
        0.708018  , 0.723561  , 0.7508731 , 0.7105566 , 0.70973957,
        0.7076567 , 0.72312844, 0.7355712 , 0.7326196 , 0.7687979 ,
        0.76779515],
       [0.77001536, 0.7643764 , 0.73267835, 0.7471203 , 0.76139486,
        0.72447705, 0.7379131 , 0.7480568 , 0.7125581 , 0.7438245 ,
        0.71193093, 0.73199654, 0.7642445 , 0.7143956 , 0.71966535,
        0.71473974, 0.7346383 , 0.74027187, 0.72643614, 0.77992934,
        0.7782748 ],
       [0.76416653, 0.78292894, 0.7424973 , 0.76028776, 0.76082027,
        0.7200955 , 0.73042154, 0.7417915 , 0.7083766

In [179]:
cos2

array([[0.64730203, 0.6325179 , 0.6395512 , 0.60721326, 0.6170237 ,
        0.56780547, 0.55887425, 0.6103828 , 0.5857983 , 0.6436064 ,
        0.64547586],
       [0.63695395, 0.6694405 , 0.6481686 , 0.6093193 , 0.6181234 ,
        0.5637252 , 0.5600145 , 0.61027676, 0.5920855 , 0.647451  ,
        0.6364725 ],
       [0.644772  , 0.64810437, 0.6618557 , 0.6158575 , 0.6220916 ,
        0.5660806 , 0.5570712 , 0.61364377, 0.5871022 , 0.6499467 ,
        0.6428431 ],
       [0.657814  , 0.6517589 , 0.66281044, 0.6516897 , 0.6394964 ,
        0.59529483, 0.5839717 , 0.63043195, 0.60640174, 0.65985626,
        0.6567203 ],
       [0.6509574 , 0.64928913, 0.652112  , 0.6246124 , 0.64132994,
        0.592481  , 0.5853598 , 0.6260165 , 0.5998993 , 0.6546117 ,
        0.64833564],
       [0.66038823, 0.6450465 , 0.6516701 , 0.6393368 , 0.6441302 ,
        0.62777853, 0.61514986, 0.64177233, 0.6182411 , 0.65933555,
        0.6600747 ],
       [0.6461258 , 0.6395905 , 0.63661265, 0.6207274 , 0.

In [180]:
cos3

array([[0.5654497 , 0.5476584 , 0.5518031 , 0.5242658 , 0.544254  ,
        0.48726243, 0.48911625, 0.537826  , 0.5131564 , 0.5591322 ,
        0.56438786],
       [0.5665155 , 0.5560533 , 0.5581775 , 0.5452387 , 0.5584916 ,
        0.5241872 , 0.52365077, 0.5528806 , 0.5318825 , 0.5660937 ,
        0.56727636],
       [0.54576635, 0.54509956, 0.53647196, 0.51917195, 0.5445626 ,
        0.49957457, 0.5064206 , 0.5341276 , 0.5130503 , 0.54834366,
        0.545914  ],
       [0.55065763, 0.5482149 , 0.54123855, 0.5261242 , 0.5461197 ,
        0.5168573 , 0.52545893, 0.5418146 , 0.53158224, 0.55302197,
        0.5524949 ],
       [0.54830027, 0.54523623, 0.54452574, 0.5219727 , 0.53858775,
        0.48486623, 0.4896215 , 0.54827034, 0.51678663, 0.55041254,
        0.54858625],
       [0.5133171 , 0.51118296, 0.5133646 , 0.48707423, 0.50417125,
        0.44232512, 0.44550148, 0.50127673, 0.47466135, 0.5177894 ,
        0.5140414 ],
       [0.53302336, 0.53001535, 0.5282191 , 0.4992582 , 0.